# ScanNet
ScanNet [Dai et al, 2017] has 3D scene dataset with following data.
<!-- - Mesh data of objects with background
- Sequence RGB and Depth image data
- Categories, camera infomation -->

Related URL
- [ScanNet Homepage](http://www.scan-net.org/index)
- [GitHub](https://github.com/ScanNet/ScanNet)



In [ ]:
%load_ext autoreload
%autoreload 2

This subsection use the following code:

In [ ]:
# path to tutlibs
import path

from tutlibs.dataset import Redwood3DScan
from tutlibs.constructing import depth_to_point
from tutlibs.transformation import Transformation as tr
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.utils import single_color

import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Reference
- [Dai, Angela, Angel X. Chang, Manolis Savva, Maciej Halber, Thomas Funkhouser, and Matthias Nießner. 2017. “Scannet: Richly-Annotated 3d Reconstructions of Indoor Scenes.” In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, 5828–39.](https://arxiv.org/abs/1702.04405)